In [1]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [2]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Projects: SLIP, SLIP Elementary, SLIP Adolescents, Clinical Imaging Genetics
project = "SLIP Adolescents"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)
    
getGraderStatusReport(name)


select * from lab.grader_table_with_metadata where grader_name = 'Jenna Schabdach';
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
Jenna Schabdach has graded 37583 unique reports of 37633 assigned where
33938 have been given a grade of 0
1771 have been given a grade of 1
1874 have been given a grade of 2


In [12]:
isOnMainTable = welcomeUser(name)

Welcome, Jenna Schabdach
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
You currently have 7 ungraded reports to work on.


In [13]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        markOneReportSQL(name, project, toHighlight)
    else:
        markSelfEvalReportSQL(name, toHighlight)


9
There are currently no reports to grade for Jenna Schabdach  in the table. Please add more to continue.


In [3]:
## If/when you run out of reports, you can uncomment the following line of code
#   and run this cell to get 50 more reports from the database. Make sure your
#   values for `project` and `queryFn` match the project you're working on.
# Projects: SLIP Adolescents, SLIP, SLIP Elementary, Clinical Imaging Genetics
# project = "SLIP Adolescents"
getSecondLookReportsToGrade(name, 50)
# getMoreReportsToGrade(name, project, numberToAdd=50)

It is expected for this function to take several minutes to run. Your patience is appreciated.
(7, 8)
Number of reports that need to be validated: 7
Number of validation reports added: 7
57 reports are in the queue for grader Jenna Schabdach
